In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

dataset = load_dataset("Samsung/samsum")
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']


tokenizer = AutoTokenizer.from_pretrained("tanmay2809/T5-base-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("tanmay2809/T5-base-samsum")
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
t5_summarizer = pipeline("summarization", model="google/flan-t5-base")


/home/tanmay/Desktop/textSummarizer/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


In [2]:
generated_summaries = [summarizer(dialogue["dialogue"], max_length=514, truncation=True)[0]["summary_text"] for dialogue in test_data]
bart_summaries = [bart_summarizer(dialogue["dialogue"], max_length=514, truncation=True)[0]["summary_text"] for dialogue in test_data]
t5_summaries = [t5_summarizer(dialogue["dialogue"], max_length=514, truncation=True)[0]["summary_text"] for dialogue in test_data]
reference_summaries = [dialogue["summary"] for dialogue in test_data]

Your max_length is set to 514, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 514, but your input_length is only 155. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)
Your max_length is set to 514, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.63 GiB of which 16.81 MiB is free. Including non-PyTorch memory, this process has 3.61 GiB memory in use. Of the allocated memory 3.40 GiB is allocated by PyTorch, and 145.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1, rouge2, rougeL = [], [], []

for ref, gen in zip(reference_summaries, generated_summaries):
    scores = scorer.score(ref, gen)
    rouge1.append(scores['rouge1'].fmeasure)
    rouge2.append(scores['rouge2'].fmeasure)
    rougeL.append(scores['rougeL'].fmeasure)

print(f"ROUGE-1: {np.mean(rouge1):.4f}")
print(f"ROUGE-2: {np.mean(rouge2):.4f}")
print(f"ROUGE-L: {np.mean(rougeL):.4f}")

ROUGE-1: 0.4627
ROUGE-2: 0.2262
ROUGE-L: 0.3635
